In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import os

train_jerry_dir = os.path.join('/content/drive/My Drive/Colab Notebooks/tomndjerry/train/jerry')

# Directory with our training human pictures
train_tom_dir = os.path.join('/content/drive/My Drive/Colab Notebooks/tomndjerry/train/tom')

train_tomandjerry_dir = os.path.join('/content/drive/My Drive/Colab Notebooks/tomndjerry/train/tomndjerry')
# Directory with our training horse pictures
validation_tom_dir = os.path.join('/content/drive/My Drive/Colab Notebooks/tomndjerry/train/tomt')

# Directory with our training human pictures
validation_jerry_dir = os.path.join('/content/drive/My Drive/Colab Notebooks/tomndjerry/test/jerryt')

validation_tomandjerry_dir = os.path.join('/content/drive/My Drive/Colab Notebooks/tomndjerry/test/tomjerryt')

train_tom_fnames = os.listdir( train_tom_dir )
train_jerry_fnames = os.listdir( train_jerry_dir )
train_tomandjerry_fnames = os.listdir( train_tomandjerry_dir )
print(train_tom_fnames[:3])
print(train_jerry_fnames[:3])
print(train_tomandjerry_fnames[:3])

['t1.jpg', 't2.jpg', 't3.png']
['j1.png', 'j2.jpg', 'j3.png']
['tj1.jpg', 'tj2.png', 'tj3.png']


In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(3, activation='softmax')
])

In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/tomndjerry/test',  
        target_size=(300, 300),  
        batch_size=4,
        
        class_mode='categorical')


validation_generator = validation_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/tomndjerry/train',  
        target_size=(300, 300), 
        batch_size=2,
        
        class_mode='categorical')

Found 12 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [7]:
history = model.fit(
      train_generator,
      steps_per_epoch=int(8/4),  
      epochs=10,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=int(12/2))

Epoch 1/10
2/2 [==============================] - 8s 4s/step - loss: 1.2646 - accuracy: 0.1250 - val_loss: 1.1402 - val_accuracy: 0.2500
Epoch 2/10
2/2 [==============================] - 2s 1s/step - loss: 1.0583 - accuracy: 0.3750 - val_loss: 1.1215 - val_accuracy: 0.4167
Epoch 3/10
2/2 [==============================] - 3s 1s/step - loss: 1.1778 - accuracy: 0.1250 - val_loss: 1.1183 - val_accuracy: 0.2500
Epoch 4/10
2/2 [==============================] - 1s 677ms/step - loss: 1.1058 - accuracy: 0.3750 - val_loss: 1.1023 - val_accuracy: 0.3333
Epoch 5/10
2/2 [==============================] - 0s 130ms/step - loss: 1.0858 - accuracy: 0.3750 - val_loss: 1.1696 - val_accuracy: 0.2500
Epoch 6/10
2/2 [==============================] - 0s 138ms/step - loss: 1.1692 - accuracy: 0.2500 - val_loss: 1.1535 - val_accuracy: 0.1667
Epoch 7/10
2/2 [==============================] - 0s 129ms/step - loss: 1.0774 - accuracy: 0.3750 - val_loss: 1.1219 - val_accuracy: 0.4167
Epoch 8/10
2/2 [=============

In [8]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [9]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(300, 300))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  classs=['tom','tomandjerry','jerry']
  print(classs[np.argmax(classes)] )

Using TensorFlow backend.


Saving to2.png to to2.png
Saving to3.png to to3.png
Saving to4.png to to4.png
[0.2669077 0.0045205 0.7285718]
jerry
[4.8629985e-08 1.0000000e+00 2.0690432e-09]
tomandjerry
[1.3572078e-04 9.9986374e-01 5.2466623e-07]
tomandjerry


In [ ]:
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my_test_model')

AttributeError: ignored